<a href="https://www.kaggle.com/code/dzung271828/ouro-trace?scriptVersionId=288557226" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Setup libraries

In [ ]:
!uv pip install --upgrade pip
!uv pip uninstall transformers tokenizers accelerate -q

!uv pip install "transformers==4.56.0" "protobuf==5.29.3" -q
!uv pip install torch datasets -q
!uv pip install pandas matplotlib seaborn tqdm wandb pyyaml
!uv pip install bitsandbytes accelerate optimum lm_eval
# !uv pip install -r requirements.txt
!uv pip install --force-reinstall --no-cache-dir "numpy<2.0"

# Suppress warnings

In [ ]:
# Suppress warnings for clean output
import warnings
import os

warnings.filterwarnings("ignore", category=UserWarning)
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"
print("✅ Packages installed successfully!")

# Install Libraries

In [ ]:
"Built-in libraries"
import re
import sys
import gc
import time
import json
import hashlib
import glob
import zipfile
from io import StringIO
from datetime import datetime
from typing import Dict, List, Optional, Tuple, Any
import yaml
import logging
import random

"Deep learning and NLP libraries"
import torch
import torch.nn.functional as F
from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    GenerationConfig,
    logging as hf_logging,
)

"Data processing libraries"
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import wandb
from tqdm.auto import tqdm
from IPython import get_ipython

# Configure logging
logging.getLogger("ContinuousBatchingLogger").setLevel(logging.ERROR)
hf_logging.set_verbosity_error()


print(f"Python Version: {sys.version}")
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA Version: {torch.version.cuda}")
!nvidia-smi

In [ ]:
import os


def configure_environment_paths():
    """Detect environment and configure paths"""
    try:
        if "google.colab" in str(get_ipython()):
            print("✅ Environment: Google Colab")
            base_data_path = "/content/"
            base_output_path = "/content/"
            environment_name = "colab"
        elif os.environ.get("KAGGLE_KERNEL_RUN_TYPE"):
            print("✅ Environment: Kaggle")
            base_data_path = "/kaggle/input/"
            base_output_path = "/kaggle/working/"
            environment_name = "kaggle"
        else:
            print("⚠️ Environment: Local/Unknown")
            base_data_path = "./data/"
            base_output_path = "./output/"
            environment_name = "local"
    except NameError:
        print("⚠️ Non-interactive session. Using local paths.")
        base_data_path = "./data/"
        base_output_path = "./output/"
        environment_name = "local"

    os.makedirs(base_output_path, exist_ok=True)
    print(f"📂 Data Path: {base_data_path}")
    print(f"📦 Output Path: {base_output_path}")

    return base_data_path, base_output_path, environment_name


INPUT_PATH, OUTPUT_PATH, ENV_NAME = configure_environment_paths()

# Setup WANDB

In [ ]:
import os
import wandb

if "colab" in ENV_NAME:
    from google.colab import userdata

    try:
        # Ensure 'WANDB_API_KEY' is the exact name in your Colab Secrets (the key icon)
        wandb_key = userdata.get("WANDB_API_KEY")
        wandb.login(key=wandb_key)
    except Exception as e:
        print(f"Could not retrieve W&B API key from Colab Secrets: {e}")

# 2. Check if running in Kaggle
elif "kaggle" in ENV_NAME:
    try:
        from kaggle_secrets import UserSecretsClient

        user_secrets = UserSecretsClient()
        wandb_key = user_secrets.get_secret("WANDB_API_KEY")
        wandb.login(key=wandb_key)
    except Exception as e:
        print(f"Could not retrieve W&B API key from Kaggle Secrets: {e}")

# Config input/output path and clone latest repo

In [ ]:
# Clone the latest github repo version
%cd {OUTPUT_PATH}
torch.cuda.empty_cache()
!rm -rf OuroTrace

In [ ]:
!git clone --branch claude https://github.com/dzungphieuluuky/OuroTrace.git
%cd OuroTrace

# Run Benchmark

In [ ]:
from src.config_loader import load_config_from_json, post_process_config
from runner import run_experiment
from src.evaluation_analysis import analyze_experiment_results

def set_all_seeds(seed):
    random.seed(seed)                          # Python random
    os.environ['PYTHONHASHSEED'] = str(seed)  # Python hash seed
    np.random.seed(seed)                      # NumPy
    torch.manual_seed(seed)                   # PyTorch CPU & GPU

    # Additional GPU-specific settings
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)      # For multi-GPU

set_all_seeds(1415)
# 1. Load Configuration from JSON
config = load_config_from_json("configs/ouro_1.4b_thinking.json")

# 2. Post-process (Convert 'torch.float16' string to object, generate timestamps)
config = post_process_config(config)

config["INFERENCE_STEPS"] = [4]
config["OPTIMIZATION"]["enable_batch"] = False
config["MODEL"]["use_torch_compile"] = True

# config["EVAL_SETTINGS"]["calculate_perplexity"] = False
# config["DATA"]["n_ary"]["num_samples_per_level"] = 0
# config["DATA"]["p_hop"]["num_samples_per_level"] = 0
# config["DATA"]["igsm"]["num_samples"] = 0
# config["DATA"]["reasoning_primitives"]["num_samples"] = 0

# 4. Execute
print("🚀 Starting Experiment...")
try:
    del model, tokenizer
    torch.cuda.empty_cache()
    gc.collect()
except:
    pass

try:
    simple_reasoning_results, ppl_results, primitives_results, benchmark_results = run_experiment(config)
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
import os
import glob
import zipfile
from typing import List
def find_result_folders(base_path: str) -> List[str]:
    """
    Return a list of absolute paths to all directories under `base_path`
    whose names start with 'results_'.
    """
    pattern = os.path.join(base_path, "results_*")
    # glob returns both files and directories; filter to directories only
    return [p for p in glob.glob(pattern) if os.path.isdir(p)]
def zip_folder(folder_path: str, output_base_path: str) -> bool:
    """
    Zip the contents of `folder_path` into a file named
    <folder_name>.zip` inside `output_base_path`.

    Returns True on success, False otherwise.
    """
    folder_name = os.path.basename(folder_path)
    zip_path = os.path.join(output_base_path, f"{folder_name}.zip")
    try:
        print(f"   -> Zipping folder: {folder_name}...")
        with zipfile.ZipFile(
            zip_path, mode="w", compression=zipfile.ZIP_DEFLATED
        ) as zipf:
            for root, _, files in os.walk(folder_path):
                for file in files:
                    full_path = os.path.join(root, file)
                    # Preserve relative path inside the zip
                    arcname = os.path.relpath(full_path, os.path.dirname(folder_path))
                    zipf.write(full_path, arcname)
        print(f"   ✅ Created ZIP: {os.path.basename(zip_path)}")
        return True
    except Exception as exc:
        print(f"   ❌ Failed to zip {folder_name}: {exc}")
        return False
def zip_stats_results_folders(output_base_path: str) -> None:
    """
    Main driver: locate all result folders and zip each one.
    """
    # Ensure the output directory exists
    os.makedirs(output_base_path, exist_ok=True)
    result_folders = find_result_folders(output_base_path)
    if not result_folders:
        print(f"⚠️ No folders starting with 'results_' found in '{output_base_path}'.")
        return
    print(f"🔍 Found {len(result_folders)} result folder(s) to zip.")
    successful = 0
    for folder in result_folders:
        if zip_folder(folder, output_base_path):
            successful += 1
    print(
        f"\n✅ DONE! Successfully zipped {successful} out of {len(result_folders)} folder(s)."
    )
if __name__ == "__main__":
    try:
        # Prefer an environment variable; fall back to a global if defined
        output_root = os.getenv("OUTPUT_PATH") or globals().get("OUTPUT_PATH")
        if not output_root:
            raise ValueError("OUTPUT_PATH not defined")
        # The script expects a sub‑folder named 'OuroTrace' under OUTPUT_PATH
        target_path = os.path.join(output_root, "")
        zip_stats_results_folders(target_path)
    except Exception as e:
        print(f"❌ An error occurred: {e}")

In [ ]:
df_simple = pd.DataFrame(simple_reasoning_results)
df_ppl = pd.DataFrame(ppl_results)

print("Final Inspection:\n")
print("Top 20 Accuracy Report:\n")
print(df_simple.head(20))
print(f"Full Response:\n")
print(df_simple["full_response"])
print("Perplexity Report:\n")
print(df_ppl.head(20))

In [ ]:
print(df_simple[["full_response", "generated_tokens"]])